In [1]:
#http://aidiary.hatenablog.com/entry/20140201/1391218771
#coding: utf-8

import numpy as np
import sys

"""
mlp.py
多層パーセプトロン
forループの代わりに行列演算にした高速化版

入力層 - 隠れ層 - 出力層の3層構造で固定（PRMLではこれを2層と呼んでいる）

隠れ層の活性化関数にはtanh関数またはsigmoid logistic関数が使える
出力層の活性化関数にはtanh関数、sigmoid logistic関数、恒等関数が使える
"""

def tanh(x):
    return np.tanh(x)

# 本来はtanh'(x) = 1 - tanh(x) ** 2であるが
# このスクリプトでは引数xがtanh(x)であることを仮定している
def tanh_deriv(x):
    return 1.0 - x ** 2

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# 本来はsigmoid'(x) = sigmoid(x) * (1 - sigmoid(x))であるが
# このスクリプトでは引数xがsigmoid(x)であることを仮定している
def sigmoid_deriv(x):
    return x * (1 - x)

def softmax(x):
    temp = np.exp(x)
    return temp / np.sum(temp)

#def softmax(x):
#    return np.exp(x)/np.sum(np.exp(x),axis=1)[:,np.newaxis]

def softmax_deriv(x):
    return softmax(x)*(np.ones(x.shape)-softmax(x))

def identity(x):
    return x

def identity_deriv(x):
    return 1

class MultiLayerPerceptron:
    def __init__(self, numInput, numHidden, numOutput, act1="tanh", act2="sigmoid"):
        """多層パーセプトロンを初期化
        numInput    入力層のユニット数（バイアスユニットは除く）
        numHidden   隠れ層のユニット数（バイアスユニットは除く）
        numOutput   出力層のユニット数
        act1        隠れ層の活性化関数（tanh or sigmoid）
        act2        出力層の活性化関数（tanh or sigmoid or identity）
        """
        # 引数の指定に合わせて隠れ層の活性化関数とその微分関数を設定
        if act1 == "tanh":
            self.act1 = tanh
            self.act1_deriv = tanh_deriv
        elif act1 == "sigmoid":
            self.act1 = sigmoid
            self.act1_deriv = sigmoid_deriv
        else:
            print "ERROR: act1 is tanh or sigmoid"
            sys.exit()

        # 引数の指定に合わせて出力層の活性化関数とその微分関数を設定
        if act2 == "tanh":
            self.act2 = tanh
            self.act2_deriv = tanh_deriv
        elif act2 == "sigmoid":
            self.act2 = sigmoid
            self.act2_deriv = sigmoid_deriv
        elif act2 == "softmax":
            self.act2 = softmax
            self.act2_deriv = softmax_deriv
        elif act2 == "identity":
            self.act2 = identity
            self.act2_deriv = identity_deriv
        else:
            print "ERROR: act2 is tanh or sigmoid or or softmax or identity"
            sys.exit()

        # バイアスユニットがあるので入力層と隠れ層は+1
        self.numInput = numInput + 1
        self.numHidden =numHidden + 1
        self.numOutput = numOutput

        # 重みを (-1.0, 1.0)の一様乱数で初期化
        self.weight1 = np.random.uniform(-1.0, 1.0, (self.numHidden, self.numInput))  # 入力層-隠れ層間
        self.weight2 = np.random.uniform(-1.0, 1.0, (self.numOutput, self.numHidden)) # 隠れ層-出力層間

    def fit(self, X, t, learning_rate=0.2, epochs=10000):
        """訓練データを用いてネットワークの重みを更新する"""
        # 入力データの最初の列にバイアスユニットの入力1を追加
        X = np.hstack([np.ones([X.shape[0], 1]), X])
        t = np.array(t)

        # 逐次学習
        # 訓練データからランダムサンプリングして重みを更新をepochs回繰り返す
        for k in range(epochs):
            #print k

            # 訓練データからランダムに選択する
            i = np.random.randint(X.shape[0])
            x = X[i]

            # 入力を順伝播させて中間層の出力を計算
            z = self.act1(np.dot(self.weight1, x))

            # 中間層の出力を順伝播させて出力層の出力を計算
            y = self.act2(np.dot(self.weight2, z))

            # 出力層の誤差を計算
            # WARNING
            # PRMLによると出力層の活性化関数にどれを用いても
            # (y - t[i]) でよいと書いてあるが
            # 下のように出力層の活性化関数の微分もかけた方が精度がずっとよくなる
            # delta2 = self.act2_deriv(y) * (y - t[i])
            #誤差関数に交差エントロピー誤差関数を利用
            delta2 = y - t[i]
            
            # 出力層の誤差を逆伝播させて隠れ層の誤差を計算
            delta1 = self.act1_deriv(z) * np.dot(self.weight2.T, delta2)

            # 隠れ層の誤差を用いて隠れ層の重みを更新
            # 行列演算になるので2次元ベクトルに変換する必要がある
            x = np.atleast_2d(x)
            delta1 = np.atleast_2d(delta1)
            self.weight1 -= learning_rate * np.dot(delta1.T, x)

            # 出力層の誤差を用いて出力層の重みを更新
            z = np.atleast_2d(z)
            delta2 = np.atleast_2d(delta2)
            self.weight2 -= learning_rate * np.dot(delta2.T, z)

    def predict(self, x):
        """テストデータの出力を予測"""
        x = np.array(x)
        # バイアスの1を追加
        x = np.insert(x, 0, 1)
        # 順伝播によりネットワークの出力を計算
        z = self.act1(np.dot(self.weight1, x))
        y = self.act2(np.dot(self.weight2, z))
        return y

#if __name__ == "__main__":
#    """XORの学習テスト"""
#    mlp = MultiLayerPerceptron(2, 2, 1, "tanh", "sigmoid")
#    X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
#    y = np.array([0, 1, 1, 0])
#    mlp.fit(X, y)
#    for i in [[0, 0], [0, 1], [1, 0], [1, 1]]:
#        print i, mlp.predict(i)

In [2]:
#coding:utf-8
import numpy as np
#from mlp import MultiLayerPerceptron
from sklearn.datasets import fetch_mldata
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import confusion_matrix, classification_report

"""
MNISTの手書き数字データの認識
scikit-learnのインストールが必要
http://scikit-learn.org/
"""

if __name__ == "__main__":
    # MNISTの数字データ
    # 70000サンプル, 28x28ピクセル
    # カレントディレクトリ（.）にmnistデータがない場合は
    # Webから自動的にダウンロードされる（時間がかかる）
    mnist = fetch_mldata('MNIST original')

    # 訓練データを作成
    X = mnist.data
    y = mnist.target

    # ピクセルの値を0.0-1.0に正規化
    X = X.astype(np.float64)
    X /= X.max()

    # 多層パーセプトロンを構築
    mlp = MultiLayerPerceptron(28*28, 100, 10, "tanh", "softmax")

    # 訓練データとテストデータに分解
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

    # 教師信号の数字を1-of-K表記に変換
    labels_train = LabelBinarizer().fit_transform(y_train)
    labels_test = LabelBinarizer().fit_transform(y_test)

    # 訓練データを用いてニューラルネットの重みを学習
    mlp.fit(X_train, labels_train, learning_rate=0.01, epochs=100000)

    # テストデータを用いて予測精度を計算
    predictions = []
    for i in range(X_test.shape[0]):
        o = mlp.predict(X_test[i])
        predictions.append(np.argmax(o))
    print confusion_matrix(y_test, predictions)
    print classification_report(y_test, predictions)

[[674   0   2   0   1  13  10   0   0   2]
 [  0 744   4   1   3   1   2   4   1   1]
 [  4   7 674   4   7   5   6   9  13   2]
 [  3   3  27 666   1  28   1   8   9   9]
 [  1   2   4   1 658   4   3   2   4  19]
 [  2   4   4  14   4 603   8   2   6   4]
 [  7   1   1   0  12   7 655   1   1   0]
 [  2   3  10   0   3   4   0 647   1  30]
 [  6   7  15  17   3  23  12   7 577  15]
 [  5   1   1   3  27   1   0  27   5 565]]
             precision    recall  f1-score   support

        0.0       0.96      0.96      0.96       702
        1.0       0.96      0.98      0.97       761
        2.0       0.91      0.92      0.92       731
        3.0       0.94      0.88      0.91       755
        4.0       0.92      0.94      0.93       698
        5.0       0.88      0.93      0.90       651
        6.0       0.94      0.96      0.95       685
        7.0       0.92      0.92      0.92       700
        8.0       0.94      0.85      0.89       682
        9.0       0.87      0.89      